# **CS 363M Final Project Spring 2025**

## Chenyi Wang, Bhuvan Kannaeganti, Suyog Valsangkar

### **Overview**

For the project in this class, you will participate in a machine learning competition where you’ll apply your ML skills to a real-world dataset. You may work individually or in teams of up to 3 students. 

The dataset for this competition comes from the Austin Animal Center, the largest no-kill animal shelter in the United States. It contains historical records of animals that have entered the shelter, including details such as species, breed, age, intake type, medical condition, and other attributes. Each animal in the dataset has a recorded outcome, which represents what eventually happened to the animal after entering the shelter.

Your goal in this competition is to build a machine learning model that predicts the final outcome of each animal admitted to the shelter, based on its intake characteristics. The possible outcomes are:

**- Adopted**: The animal was placed into a new home.<br>
**- Return to Owner**: The animal was reclaimed by its original owner.<br>
**- Euthanasia**: The animal was humanely euthanized due to medical or behavioral concerns.<br>
**- Died**: The animal passed away while in the shelter’s care.<br>
**- Transfer**: The animal was moved to another shelter or rescue organization.<br>

By accurately predicting these outcomes, your model can help identify factors that influence an animal's journey through the shelter system and potentially aid in improving adoption and survival rates, shelter policies, or allocation of resources.


## **Code and Analysis Below:**

1. We need to go through the dataset and examine the existing features for patterns and methods we can feature engineer our data to enhance our final predictions. 

In [385]:
import pandas as pd

animal_data = pd.read_csv('train.csv')
animal_data.sample(5) # sample some data

,Id,Name,Intake Time,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Outcome Time,Date of Birth,Outcome Type
8119,A844525,A844525,10/16/2021 05:21:00 PM,Konstanty Circle in Austin (TX),Stray,Neonatal,Cat,Unknown,1 week,Domestic Shorthair,Black,10/16/2021 06:18:00 PM,10/09/2021,Transfer
95160,A696878,Bodie,02/13/2015 05:54:00 PM,Barton Creek Square in Austin (TX),Stray,Normal,Dog,Intact Male,2 months,Cocker Spaniel/Australian Shepherd,Brown Brindle/Brown Brindle,02/17/2015 06:41:00 PM,12/13/2014,Adoption
48353,A750311,Yoda,05/25/2017 02:42:00 PM,10300 N Ih 35 in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Miniature Poodle Mix,White,05/29/2017 05:29:00 PM,05/25/2016,Adoption
54261,A837471,NaN,06/23/2021 12:42:00 PM,1156 W Cesar Chavez St in Austin (TX),Stray,Medical,Dog,Intact Male,1 month,Manchester Terrier,Black/Brown,06/26/2021 11:21:00 AM,04/23/2021,Transfer
32810,A765782,Harley,01/26/2018 05:02:00 PM,2700 Guadalupe Street in Austin (TX),Stray,Normal,Dog,Unknown,2 years,Labrador Retriever Mix,Black/White,01/26/2018 06:20:00 PM,01/26/2016,Return to Owner


2. **Data Cleaning**: After observing the initial dataset, I see that we need to perform data cleaning. We need to separate the sex upon intake feature as it currently also includes sterilization health status. We also need to fix the age upon intake column as it is using all sorts of time units. The name feature also does not exist within the test set, we need to be careful and only train the model on features that exist within the test set. This also applies to Id, which does not correlate to the Id within the test set. Therefore, we can also eliminate name and id before training the model.

In [386]:
# drop name feature entirely
animal_data.drop("Name", axis=1, inplace=True)

# drop id feature entirely
animal_data.drop("Id", axis=1, inplace=True)

# drop outcome time
animal_data.drop("Outcome Time", axis=1, inplace=True)

# drop birth date
animal_data.drop("Date of Birth", axis=1, inplace=True)

# transform Animal Type into a binary column: Cat -> True, Dog -> False
animal_data['Cat'] = animal_data['Animal Type'].apply(lambda x: True if x == 'Cat' else False)
animal_data.drop('Animal Type', axis=1, inplace=True)

# separate age at intake and reproductive status, create new column
split_cols = animal_data['Sex upon Intake'].str.split(' ', n=1, expand=True)

# handle cases where the split returns "Unknown"
animal_data['Sterilized'] = split_cols[0].map({
    'Neutered': 'True',
    'Spayed': 'True',
    'Intact': 'False',
    'Unknown': 'False'
}).fillna('False')  # unexpected values, let's assume not sterilized

# assign gender, we can keep it a binary by make the feature "Male", where male = true and female = false
animal_data['Male'] = split_cols[1].apply(lambda x: False if x == 'Female' else True)
animal_data.drop("Sex upon Intake", axis=1, inplace=True)

# helper to convert age upon intake to years
def age_to_years(age_str):
    if pd.isna(age_str):
        return None
    
    number, unit = age_str.split()
    number = float(number)
    
    if "year" in unit:
        return number
    elif "month" in unit:
        return number / 12
    elif "week" in unit:
        return number / 52
    elif "day" in unit:
        return number / 365
    else:
        return None  # in case of an unexpected format

# convert age to years
animal_data['Age'] = animal_data['Age upon Intake'].apply(age_to_years)
animal_data.drop('Age upon Intake', axis=1, inplace=True)

pd.set_option('display.max_columns', None)
animal_data.sample(5) # sample some data after transformations

,Intake Time,Found Location,Intake Type,Intake Condition,Breed,Color,Outcome Type,Cat,Sterilized,Male,Age
59900,05/10/2024 01:33:00 PM,Austin (TX),Owner Surrender,Normal,Miniature Schnauzer,Black/White,Adoption,False,True,False,6.000000
29235,04/28/2018 01:27:00 PM,10900 North Lamar Boulevard in Austin (TX),Stray,Normal,Labrador Retriever Mix,Black/White,Return to Owner,False,True,False,0.750000
51481,11/08/2013 11:00:00 PM,12Th in Austin (TX),Public Assist,Normal,Labrador Retriever Mix,Black/White,Return to Owner,False,True,True,6.000000
73021,06/17/2019 05:52:00 PM,Montopolis And Vargas in Austin (TX),Stray,Normal,Great Pyrenees Mix,White/Black,Adoption,False,False,True,1.000000
54332,05/20/2024 04:49:00 PM,Burrell Dr in Austin (TX),Stray,Normal,Domestic Shorthair,White,Adoption,True,False,False,0.083333


3. **Feature Engineering**: We need to one hot encode the categorical features. However, there are categoricals that have many labels, some with very little data points. We can study the trend of their similarities to each other and group to reduce dimensionality.

Let's look at Intake Condition and see all the possible labels for that class.

In [387]:
# Count the occurrences of each intake condition label
intake_condition_counts = animal_data['Intake Condition'].value_counts()

# Print the counts for each intake condition
for condition, count in intake_condition_counts.items():
    print(f"{condition}: {count}")

Normal: 95010
Injured: 6394
Sick: 4295
Nursing: 2957
Neonatal: 1240
Aged: 373
Medical: 298
Other: 247
Pregnant: 111
Feral: 104
Med Attn: 48
Behavior: 42
Unknown: 12
Neurologic: 10
Med Urgent: 7
Parvo: 5
Space: 2
Agonal: 1
Congenital: 1


We see above that there are 19 different categorical values for the Intake Condition feature, we can merge some of these rarer classification together.

Let's explore the outcome percentages of each intake condition so we can group these conditions better and reduce the amount of labels we need to one hot encode (increases dimensionality).

In [388]:
# Iterate through each unique intake condition and calculate outcome percentages
intake_conditions = animal_data['Intake Condition'].unique()

for condition in intake_conditions:
    condition_data = animal_data[animal_data['Intake Condition'] == condition]
    total_count = len(condition_data)
    if total_count > 0:
        outcome_percentages = condition_data['Outcome Type'].value_counts(normalize=True) * 100
        print(f"Intake Condition: {condition}")
        print(outcome_percentages)
        print("-" * 50)
    else:
        print(f"Intake Condition: {condition} has no entries.")
        print("-" * 50)

Intake Condition: Normal
Outcome Type
Adoption           52.596569
Transfer           29.360067
Return to Owner    15.985686
Euthanasia          1.513525
Died                0.544153
Name: proportion, dtype: float64
--------------------------------------------------
Intake Condition: Injured
Outcome Type
Adoption           34.813888
Transfer           30.669378
Euthanasia         18.720676
Return to Owner    12.605568
Died                3.190491
Name: proportion, dtype: float64
--------------------------------------------------
Intake Condition: Pregnant
Outcome Type
Transfer           54.954955
Adoption           39.639640
Return to Owner     4.504505
Died                0.900901
Name: proportion, dtype: float64
--------------------------------------------------
Intake Condition: Neonatal
Outcome Type
Transfer           69.919355
Adoption           25.564516
Died                3.064516
Return to Owner     0.967742
Euthanasia          0.483871
Name: proportion, dtype: float64
-------

Groupings based on similar outcome percentages

In [389]:
'''
   [Group]	       [Categories]
    Normal	        Normal
    Med_Minor	    Injured, Medical (more similar outcome adopt %)
    Nursing 	    Nursing, Neonatal
    Med_Major       Med Attn, Med Urgent, Neurologic, Pregnant, Sick (more similar outcome transfer %)
    Behavioral	    Feral, Behavior
    Critical    	Agonal, Aged, Congenital, Parvo, Space, Other, Unknown
'''

'\n   [Group]\t       [Categories]\n    Normal\t        Normal\n    Med_Minor\t    Injured, Medical (more similar outcome adopt %)\n    Nursing \t    Nursing, Neonatal\n    Med_Major       Med Attn, Med Urgent, Neurologic, Pregnant, Sick (more similar outcome transfer %)\n    Behavioral\t    Feral, Behavior\n    Critical    \tAgonal, Aged, Congenital, Parvo, Space, Other, Unknown\n'

We see that our merged class labels have relatively similar outcomes percentage-wise. This will reduce dimensionality and improve our training whilst not losing the originality of the groupings.

In [390]:
# intake condition into grouped categories
condition_map = {
    'Normal': 'Normal',
    'Injured': 'Med_Minor',
    'Sick': 'Med_Major',
    'Nursing': 'Nursing',
    'Neonatal': 'Nursing',
    'Med Attn': 'Med_Major',
    'Med Urgent': 'Med_Major',
    'Medical': 'Med_Minor',
    'Neurologic': 'Med_Major',
    'Pregnant': 'Med_Major',
    'Feral': 'Behavioral',
    'Behavior': 'Behavioral',
}

# map with a fallback to 'Rare'
animal_data['Condition'] = animal_data['Intake Condition'].map(condition_map).fillna('Rare')
animal_data = pd.get_dummies(animal_data, columns=['Condition'])
animal_data.drop('Intake Condition', axis=1, inplace=True)

animal_data.sample(5)  # sample some data after transformation

,Intake Time,Found Location,Intake Type,Breed,Color,Outcome Type,Cat,Sterilized,Male,Age,Condition_Behavioral,Condition_Med_Major,Condition_Med_Minor,Condition_Normal,Condition_Nursing,Condition_Rare
110111,10/02/2015 02:19:00 PM,Manor Rd & Cherrywood in Austin (TX),Stray,Great Pyrenees Mix,White,Return to Owner,False,False,True,1.000000,False,False,False,True,False,False
45160,07/18/2018 02:37:00 PM,905 Bedford Street in Austin (TX),Stray,Chihuahua Shorthair/Cardigan Welsh Corgi,Tan,Adoption,False,False,True,0.333333,False,False,False,True,False,False
3106,09/10/2017 12:02:00 PM,2337 East Cesar Chavez Street in Austin (TX),Stray,Staffordshire Mix,Tan/White,Adoption,False,True,True,2.000000,False,False,False,True,False,False
102109,07/22/2020 08:04:00 AM,7325 Schebler St in Travis (TX),Stray,Labrador Retriever/Pug,Black,Return to Owner,False,False,True,1.000000,False,False,False,True,False,False
50204,08/05/2015 03:57:00 PM,6902 Deep Cir #A in Austin (TX),Owner Surrender,Labrador Retriever Mix,Black/White,Adoption,False,True,True,4.000000,False,False,False,True,False,False


Let's do the same for Intake Type and check the outcome percentages of each condition.

In [391]:
# observe outcome percentages for each intake type so we can group these categoricals, reduces dimensionality
intake_types = animal_data['Intake Type'].unique()

for intake_type in intake_types:
    type_data = animal_data[animal_data['Intake Type'] == intake_type]
    total_count = len(type_data)
    if total_count > 0:
        outcome_percentages = type_data['Outcome Type'].value_counts(normalize=True) * 100
        print(f"Intake Type: {intake_type}")
        print(outcome_percentages)
        print("-" * 50)
    else:
        print(f"Intake Type: {intake_type} has no entries.")
        print("-" * 50)

Intake Type: Stray
Outcome Type
Adoption           47.774546
Transfer           34.219783
Return to Owner    13.905286
Euthanasia          3.056120
Died                1.044266
Name: proportion, dtype: float64
--------------------------------------------------
Intake Type: Public Assist
Outcome Type
Return to Owner    63.545914
Adoption           18.528788
Transfer           14.222802
Euthanasia          3.262111
Died                0.440385
Name: proportion, dtype: float64
--------------------------------------------------
Intake Type: Owner Surrender
Outcome Type
Adoption           64.671050
Transfer           26.502541
Return to Owner     5.399357
Euthanasia          2.736980
Died                0.690073
Name: proportion, dtype: float64
--------------------------------------------------
Intake Type: Abandoned
Outcome Type
Adoption           63.017032
Transfer           26.845093
Return to Owner     9.083536
Euthanasia          0.648824
Died                0.405515
Name: proportion, 

These would be the groupings that are most similar to each other based on the outcome percentages.

In [392]:
'''
    [Group]	         [Categories]
     Public Assist    Public Assist
     Stray            Stray, Wildlife
     Owner-Initiated  Abandoned, Owner Surrender
     Euthanasia       Euthanasia Request
'''

'\n    [Group]\t         [Categories]\n     Public Assist    Public Assist\n     Stray            Stray, Wildlife\n     Owner-Initiated  Abandoned, Owner Surrender\n     Euthanasia       Euthanasia Request\n'

In [393]:
# Define the mapping for grouping intake types
intake_type_map = {
    'Stray': 'Stray',
    'Public Assist': 'Public Assist',
    'Wildlife': 'Stray',
    'Abandoned': 'Owner Initiated',
    'Owner Surrender': 'Owner Initiated',
    'Euthanasia Request': 'Euthanasia'
}

# Map the intake types to their respective groups
animal_data['Intake'] = animal_data['Intake Type'].map(intake_type_map).fillna('Other')
animal_data = pd.get_dummies(animal_data, columns=['Intake'])
animal_data.drop('Intake Type', axis=1, inplace=True)

animal_data.sample(5)  # sample some data after transformation


,Intake Time,Found Location,Breed,Color,Outcome Type,Cat,Sterilized,Male,Age,Condition_Behavioral,Condition_Med_Major,Condition_Med_Minor,Condition_Normal,Condition_Nursing,Condition_Rare,Intake_Euthanasia,Intake_Owner Initiated,Intake_Public Assist,Intake_Stray
9989,01/13/2018 05:29:00 PM,Spicewood Springs And Mesa in Austin (TX),Pointer Mix,Black/White,Return to Owner,False,False,False,2.000000,False,False,False,True,False,False,False,False,False,True
95323,10/06/2015 11:57:00 AM,Austin (TX),Domestic Shorthair Mix,Brown Tabby,Adoption,True,False,False,0.166667,False,False,False,True,False,False,False,True,False,False
54756,02/20/2018 02:56:00 PM,Gattis School Road And Tollroad 130 in Pfluger...,Domestic Shorthair Mix,Calico,Transfer,True,True,False,2.000000,False,False,False,True,False,False,False,False,False,True
34444,08/21/2014 01:23:00 PM,13904 Old Hwy 20 in Manor (TX),Domestic Shorthair Mix,Black/White,Transfer,True,False,True,0.076923,False,False,False,True,False,False,False,False,False,True
49979,09/14/2023 01:40:00 PM,14311 Wells Port Drive in Austin (TX),Domestic Shorthair,Brown/White,Adoption,True,False,True,0.083333,False,False,True,False,False,False,False,False,False,True


Still need to find a way to encode:
- Location Found
- Breed
- Color

How do they even affect the outcome or is there even a correlation? Are some colors / breeds more or less desirable than others (therefore changing adoption rates)?